In [6]:
import numpy as np
import pandas as pd
import torch_geometric
import networkx as nx
from torch_geometric.data import Data, Dataset
from torch_geometric.datasets.planetoid import Planetoid
from torch_geometric.transforms.to_undirected import ToUndirected
import torch

In [102]:
edge_list = pd.read_csv('/home/jrm28/fairness/subgraph_sketching-original/dataset/gplus/raw/100637660947564674695.edges', sep=' ', header=None)

In [103]:
edge_list.sort_values(by=[0])

,0,1
3926,100103314581848288036,116995710336756123292
2755,100103314581848288036,108981679893410895534
184,100103314581848288036,111033676716092524756
3031,100103314581848288036,110527332382139046457
422,100103314581848288036,102062087222207096193
...,...,...
2494,118391161680564572266,111799466965072047882
2191,118391161680564572266,112943946047332492941
1199,118391161680564572266,100624241693398887245
2863,118391161680564572266,101848191156408080085


In [104]:
unique_values = edge_list.stack().unique()
unique_values_sorted = np.sort(unique_values)
node_mapper = dict(zip(unique_values_sorted, range(len(unique_values_sorted))))

In [105]:
edge_list[0] = edge_list[0].map(node_mapper)

In [106]:
edge_list[1] = edge_list[1].map(node_mapper)

In [107]:
edge_array = edge_list.sort_values(by=[0, 1]).values

In [108]:
features = np.loadtxt('/home/jrm28/fairness/subgraph_sketching-original/dataset/gplus/raw/100637660947564674695.feat')

# sort features by node id
sorted_idx = features[:, 0].argsort()
features = features[sorted_idx, :]

In [110]:
transform = ToUndirected()

# feature 1 will be the sensitive attribute
x = torch.tensor(features[:, 1:], dtype=torch.float32)
y = x[:, 1]

# Removing columns 1, 2, and 3 (sensitive attributes)
x = x[:, 4:]

gplus = Data(x=x, y=y, edge_index=torch.tensor(edge_array.T))
gplus = transform(gplus)

In [111]:
torch.save(gplus, '/home/jrm28/fairness/subgraph_sketching-original/dataset/gplus/processed/gplus_100637660947564674695.pt')

## EDA

In [92]:
torch.vstack((y[gplus.edge_index[0]], y[gplus.edge_index[1]])).sum(axis=0)  

tensor([1., 2., 1.,  ..., 0., 1., 1.])

In [94]:
(torch.vstack((y[gplus.edge_index[0]], y[gplus.edge_index[1]])).sum(axis=0) == 0).sum() + (torch.vstack((y[gplus.edge_index[0]], y[gplus.edge_index[1]])).sum(axis=0) == 2).sum(), (torch.vstack((y[gplus.edge_index[0]], y[gplus.edge_index[1]])).sum(axis=0) == 1).sum(), 

(tensor(233290), tensor(70242))